<a href="https://colab.research.google.com/github/player1537/Train-Bloom-560m/blob/main/Train_Bloom_560m_Full_%26_LoRA_on_Dolphin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import annotations
%pip install --quiet --upgrade pip
%pip install --upgrade --force-reinstall git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git
# try:
#     __import__('importlib').reload(__import__('sys').modules['mediocreatbest'])
# except KeyError:
#     print(f'Module "mediocreatbest" not in sys.modules?')
from mediocreatbest import auto, run
auto.register('mediocreatbest', 'git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.7 MB/s eta 0:00:00
  Cloning https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git to /tmp/pip-req-build-l0lhie20
  Running command git clone --filter=blob:none --quiet https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git /tmp/pip-req-build-l0lhie20
  Resolved https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git to commit 47916d891d0fbbf7c42bdae924ce81bdd56d75d8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mediocreatbest: filename=mediocreatbest-0.1.1-py3-none-any.whl size=4767 sha256=653219ac47f8ea05ab3cb39a25ede090f15402296608f487bf9111c855b54490
  Stored in directory: /tmp/pip-ephem-wheel-cache-c30fqvxw/wheels/cc/6d/dd/d4137c3485df0cb8773c136ecc231252590bf217127892aaa5
Successfully built mediocreatbest


In [2]:
#@title HuggingFace Login
auto.huggingface_hub.notebook_login()

In [2]:
#@title Configuration
DO_PREPROCESS = False    #@param {"type":"boolean"}
DO_TRAIN = False    #@param {"type":"boolean"}
DO_LORA = True  #@param {"type":"boolean"}
DO_GOOGLE_DRIVE = False  #@param{type:"boolean"}
DO_EVALUATE = True  #@param{type:"boolean"}

# Output Config
CHECKPOINT_DIRECTORY = (
    'checkpoints'
)
MY_DATASET_NAME = (
    'player1537/Bloom-560m-trained-on-Dolphin'
)
MY_MODEL_NAME = MY_REPOSITORY_NAME = (
    'player1537/Bloom-560m-LoRA-trained-on-Dolphin'
) if DO_LORA else (
    'player1537/Bloom-560m-Full-trained-on-Dolphin'
)

# Input Config
BASE_MODEL_NAME = (
    'bigscience/bloom-560m'
)
PEFT_MODEL_NAME = (
    # None
    MY_MODEL_NAME
) if DO_LORA else (
    None
)
TOKENIZER_NAME = (
    BASE_MODEL_NAME
)
DATASET_NAME = (
    'ehartford/dolphin'
    # 'ehartford/wizard_vicuna_70k_unfiltered'
    # 'player1537/wizard-vicuna-unfiltered-summarized-70k'
)

# Training Config
SEED = (
    1337
)
LORA_ALPHA = (
    32
)
LORA_DROPOUT = (
    0.1
)
SHARD = 2,20 #@param {"type":"raw"} ["0,20","1,20","2,20","3,20","4,20","5,20","6,20","7,20","8,20","9,20","10,20","11,20","12,20","13,20","14,20","15,20","16,20","17,20","18,20","19,20"]
CONTEXT_SIZE = 1024 #@param {"type":"raw"} [256,512,768,1024,1280,1536,1792,2048]
MINIBATCH_SIZE = 1 #@param {"type":"raw"} [1,2,3,4]
LEARNING_RATE = (
    2e-5
)

In [4]:
@run(cond=DO_GOOGLE_DRIVE)
def __mount_drive():
    root = auto.pathlib.Path('/content/drive')
    auto.google.colab.drive.mount(str(root))

    dst = root / 'MyDrive' / MY_REPOSITORY_NAME
    dst.mkdir(parents=True, exist_ok=True)

    src = auto.pathlib.Path.cwd() / CHECKPOINT_DIRECTORY
    if not src.exists():
        src.symlink_to(dst)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocess

In [5]:
@run(cond=DO_PREPROCESS)
def tokenizer():
    return auto.transformers.AutoTokenizer.from_pretrained(
        TOKENIZER_NAME,
        add_prefix_space=True,
    )

In [6]:
@run(cond=DO_PREPROCESS)
def __download_dataset():
    DATASET_LOCAL_DIRECTORY = (
        DATASET_NAME.replace('/', '--')
    )

    repository = auto.huggingface_hub.Repository(
        local_dir=DATASET_LOCAL_DIRECTORY,
        clone_from=DATASET_NAME,
        repo_type='dataset',
        # revision='67f8c7a87db4fe0b985f317f7330c664e8122cfe',
        skip_lfs_files=True,
    )

    auto.subprocess.run([
        'git',
        '-C', DATASET_LOCAL_DIRECTORY,
        'lfs', 'pull',
        '--include', 'flan1m-alpaca-uncensored.jsonl',
    ], check=True)

    auto.subprocess.run([
        'ls',
        '-lah',
        DATASET_LOCAL_DIRECTORY,
    ])

In [7]:
@run(cond=DO_PREPROCESS, after=lambda dataset: print(dataset, dataset[0]))
def dataset():
    return auto.datasets.load_dataset(
        DATASET_NAME,
        data_files='flan1m-alpaca-uncensored.jsonl',
        split='train',
    )

In [8]:
# auto.pprint.pp(g['dataset'][1])

In [3]:
@auto.mediocreatbest.doctest
def __text(datum: Dict[Any, Any]=None, /, **kwargs) -> str:
    r"""

    >>> __text({
    ...   "instruction": "Test instruction.",
    ...   "input": "Test input.",
    ...   "output": "Test output.",
    ... })
    '<s>INSTRUCTION: Test instruction. INPUT: Test input. OUTPUT: Test output.</s>'

    >>> __text({
    ...   "instruction": "Test instruction.",
    ...   "input": "Test input.",
    ...   "output": None,
    ... })
    '<s>INSTRUCTION: Test instruction. INPUT: Test input. OUTPUT:'

    """

    if datum is None:
        datum = kwargs

    return (
        f"""<s>"""
        f"""INSTRUCTION: {datum['instruction']} """
        f"""INPUT: {datum['input']} """
        f"""OUTPUT: {datum['output']}</s>"""
    ) if datum.get('output', None) is not None else (
        f"""<s>"""
        f"""INSTRUCTION: {datum['instruction']} """
        f"""INPUT: {datum['input']} """
        f"""OUTPUT:"""
    )


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



In [10]:
@run(cond=DO_PREPROCESS, after=lambda dataset: print(dataset, dataset[0]))
def dataset(dataset, tokenizer, /):
    nchars = []
    ntokens = []
    for i, datum in enumerate(dataset):
        if i > 1000: break

        # print(datum)
        text = __text(datum)
        nchar = len(text)
        ntoken = len(tokenizer.tokenize(text))

        nchars.append(nchar)
        ntokens.append(ntoken)

    result = auto.scipy.stats.linregress(nchars, ntokens)
    print(f'ntoken = {result.slope:0.3f} * nchar + {result.intercept:0.3f} (r = {result.rvalue:0.3f})')

    @auto.functools.singledispatch
    def __ntoken(_, /):
        raise NotImplementedError()

    @__ntoken.register
    def __ntoken_from_nchar(nchar: int, /) -> int:
        return int(result.slope * nchar + result.intercept)

    @__ntoken.register
    def __ntoken_from_text(text: str, /) -> int:
        return __ntoken(len(text))

    @__ntoken.register
    def __ntoken_from_datum(datum: dict, /) -> int:
        return __ntoken(__text(datum))

    # counter = auto.collections.Counter()
    # for i, datum in enumerate(dataset):
    #     if i > 10000: break

    #     ntoken = __ntoken(datum)
    #     ntoken = ntoken // 10 * 10
    #     counter[ntoken] += 1

    # lo = min(counter.keys())
    # hi = max(counter.keys())
    # # print(counter.most_common(10))
    # largest = counter.most_common(1)[0][1]

    # for i in range(lo, hi+1, 10):
    #     count = counter.get(i, 0)
    #     count = count * 80
    #     if count > 0:
    #         count = 1 + count // largest

    #     print(f'{i:03d}t ' + '#' * count)

    def cond(datum) -> bool:
        return __ntoken(datum) <= 512

    before = len(dataset)
    dataset = dataset.filter(cond)
    after = len(dataset)

    print(f'From {before:,d} to {after:,d} ({before-after:,d} removed)')

    return dataset

In [11]:
@run(cond=DO_PREPROCESS, after=lambda dataset: print(dataset, dataset[0]))
def dataset(dataset, /):
    """Convert each instruction into text"""

    def each(datum):
        return { "text": __text(datum) }

    return dataset.map(each)

In [12]:
@run(cond=DO_PREPROCESS, after=lambda dataset: auto.pprint.pp(print(dataset, dataset[0])))
def dataset(dataset, tokenizer, /):
    def each(datum):
        text = datum['text']
        input_ids = tokenizer.encode(text)
        return { 'input_ids': input_ids }

    return dataset.map(each)

In [13]:
@run(cond=DO_PREPROCESS)
def __push_dataset_to_hub(dataset, /):
    got = set(dataset[0].keys())
    exp = { 'instruction', 'input', 'output', 'text', 'tokens' }
    for key in exp:
        assert key in got, \
            f"""Expected dataset to have key: {key=!r}"""
        got.pop(key)
    assert len(got) == 0, \
        f"""Unexpected key in dataset: {got=!r}"""

    dataset.push_to_hub(
        MY_DATASET_NAME,
    )

# Train

In [14]:
@run(cond=DO_TRAIN)
def dataset():
    return auto.datasets.load_dataset(
        MY_DATASET_NAME,
        split='train',
    )

In [15]:
@run(cond=DO_TRAIN)
def __print_dataset_statistics(dataset, /):
    got = set(dataset[0].keys())
    exp = { 'instruction', 'input', 'output', 'text', 'input_ids' }
    for key in exp:
        assert key in got, \
            f"""Expected dataset to have key: {key=!r}"""
        got.remove(key)
    assert len(got) == 0, \
        f"""Unexpected key in dataset: {got=!r}"""

    N = len(dataset)
    print(f'{N = :,d}')

    ntokens = 0
    for i, datum in enumerate(dataset):
        if i >= 1000: break
        ntokens += len(datum['input_ids'])

    estimate = ntokens * N // 1000
    print(f'{estimate = :,d}')

N = 694,524
estimate = 180,211,614


In [16]:
@run(cond=DO_TRAIN, after=print)
def dataset(dataset, /):
    return dataset.shard(max(SHARD), min(SHARD), contiguous=True)

Dataset({
    features: ['instruction', 'input', 'output', 'text', 'input_ids'],
    num_rows: 34727
})


In [17]:
@run(cond=DO_TRAIN, after=print)
def dataset(dataset, /):
    def each(datums: dict[str, list]) -> dict[str, list]:
        def emit(done=False, /, *, input_ids=None, datums={}):
            if done: return datums
            datums.setdefault('input_ids', []).append(input_ids)

        it = datums['input_ids']
        it = auto.more_itertools.collapse(it)
        it = auto.more_itertools.chunked(it, n=CONTEXT_SIZE)
        it = (x for x in it if len(x) == CONTEXT_SIZE)
        for input_ids in it:
            emit(
                input_ids=input_ids,
            )

        return emit(True)

    return dataset.map(
        each,
        batched=True,
        remove_columns=['instruction', 'input', 'output', 'text', 'input_ids'],
    )

Parameter 'function'=<function dataset.<locals>.each at 0x790dafbfe710> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/34727 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids'],
    num_rows: 8617
})


In [18]:
@run(cond=DO_TRAIN)
def model():
    auto.xformers

    return auto.transformers.AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        torch_dtype=(
            # auto.torch.float16
            # if auto.torch.cuda.is_available() else
            auto.torch.float32
        ),
    )

@run(cond=DO_TRAIN and DO_LORA and PEFT_MODEL_NAME is None)
def model(model, /):
    return auto.peft.get_peft_model(
        model,
        auto.peft.LoraConfig(
            task_type='CAUSAL_LM',
            lora_alpha=LORA_ALPHA,
            lora_dropout=LORA_DROPOUT,
            inference_mode=False,
        ),
    )

@run(cond=DO_TRAIN and DO_LORA and PEFT_MODEL_NAME is not None)
def model(model, /):
    return auto.peft.PeftModel.from_pretrained(
        model,
        PEFT_MODEL_NAME,
        is_trainable=True,
    )

@run(cond=DO_TRAIN and DO_LORA)
def __print_model(model, /):
    model.print_trainable_parameters()

trainable params: 786,432 || all params: 560,001,024 || trainable%: 0.14043402892063284


In [19]:
# @title  { form-width: "256px" }
# @title  { form-width: "10%" }
@run(cond=DO_TRAIN)
def trainer(model, dataset, /):
    BATCH_PER_SECOND = 1.38 #@param {"type":"number"}
    MINUTES_PER_LOG = 2.5  #@param {type:"number"}
    MINUTES_PER_SAVE = 10  #@param {type:"number"}

    @auto.mediocreatbest.immediate
    def __print_step_information():
        N = len(dataset)
        x = []
        for i in range(0, N, int(MINUTES_PER_SAVE*60 * BATCH_PER_SECOND)):
            x.append((i, 'save'))
        for i in range(0, N, int(MINUTES_PER_LOG*60 * BATCH_PER_SECOND)):
            x.append((i, 'log'))
        for step, name in sorted(x):
            print(f'{step:,d} {name}')

    def p(name, value, /):
        print(f'{name} = {value!r}')
        return value

    class Trainer(auto.transformers.Trainer):
        def compute_loss(self, model, inputs, return_outputs=False):
            return model(
                input_ids=inputs["input_ids"],
                attention_mask=auto.torch.ones_like(inputs["input_ids"]).bool(),
                labels=inputs["input_ids"],
            ).loss

    return Trainer(
        model=model,
        train_dataset=dataset,
        args=auto.transformers.TrainingArguments \
            ( learning_rate=LEARNING_RATE
            , seed=SEED
            , output_dir=CHECKPOINT_DIRECTORY
            , per_device_train_batch_size=MINIBATCH_SIZE
            , per_device_eval_batch_size=MINIBATCH_SIZE

            , save_steps=p('save_steps', (
                int(MINUTES_PER_SAVE*60 * BATCH_PER_SECOND)
            ))
            , logging_steps=p('logging_steps', (
                int(MINUTES_PER_LOG * 60 * BATCH_PER_SECOND)
            ))

            , overwrite_output_dir=True    #@param {type:"boolean"}
            , evaluation_strategy="no"    #@param ["no", "steps", "epochs"] {"type": "string"}
            , num_train_epochs=1    #@param {"type": "integer"}
            , logging_strategy="steps"    #@param ["no", "steps", "epochs"] {"type": "string"}
            , logging_first_step=True    #@param {type:"boolean"}
            , save_strategy="steps"    #@param ["no", "steps", "epochs"] {"type": "string"}
            , save_total_limit=3    #@param {"type": "integer"}
            , save_safetensors=False    #@param {type:"boolean"}
            , gradient_accumulation_steps=1
            , fp16=True    #@param {type:"boolean"}
            , fp16_full_eval=True    #@param {type:"boolean"}
            # , push_to_hub=True    #@param {type:"boolean"}
            # , hub_model_id=MY_REPOSITORY_NAME
            # , hub_strategy='checkpoint'    #@param ["end", "every_save", "checkpoint", "all_checkpoints"] {"type": "string"}
            , auto_find_batch_size=False    #@param {type:"boolean"}
        ),
    )

0 log
0 save
206 log
412 log
618 log
824 log
827 save
1,030 log
1,236 log
1,442 log
1,648 log
1,654 save
1,854 log
2,060 log
2,266 log
2,472 log
2,481 save
2,678 log
2,884 log
3,090 log
3,296 log
3,308 save
3,502 log
3,708 log
3,914 log
4,120 log
4,135 save
4,326 log
4,532 log
4,738 log
4,944 log
4,962 save
5,150 log
5,356 log
5,562 log
5,768 log
5,789 save
5,974 log
6,180 log
6,386 log
6,592 log
6,616 save
6,798 log
7,004 log
7,210 log
7,416 log
7,443 save
7,622 log
7,828 log
8,034 log
8,240 log
8,270 save
8,446 log
save_steps = 827
logging_steps = 206


In [20]:
# Step 	Training Loss
# 1 	2.620200
# 206 	2.264300
# 412 	2.264700
# 618 	2.259600
# 824 	2.234900
# 1030 	2.258500
# 1236 	2.306800
# 1442 	2.247800
# 1648 	2.230700
# 1854 	2.244000
# 2060 	2.242700
# 2266 	2.248900
# 2472 	2.232800
# 2678 	2.268700
# 2884 	2.214100
# 3090 	2.219500
# 3296 	2.237900
# 3502 	2.225200
# 3708 	2.245800
# 3914 	2.258400
# 4120 	2.229800
# 4326 	2.214100
# 4532 	2.252800
# 4738 	2.275200
# 4944 	2.253600
# 5150 	2.208200
# 5356 	2.229100
# 5562 	2.236700
# 5768 	2.235700
# 5974 	2.226100
# 6180 	2.202500
# 6386 	2.216700
# 6592 	2.203600
# 6798 	2.186300
# 7004 	2.206600
# 7210 	2.170400
# 7416 	2.204700
# 7622 	2.162700
# 7828 	2.215300
# 8034 	2.184700
# 8240 	2.240600
# 8446 	2.196600

@run(cond=DO_TRAIN)
def __train(trainer, /):
    trainer.train(
        resume_from_checkpoint=(
            # 'checkpoints/last-checkpoint'
            # 'checkpoints/checkpoint-4962'
            # 'checkpoint-3308'
            # 'tmp/last-checkpoint'
            False
        ),
    )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,2.620200
206,2.264300
412,2.264700
618,2.259600
824,2.234900
1030,2.258500
1236,2.306800
1442,2.247800
1648,2.230700
1854,2.244000


In [21]:
@run(cond=DO_TRAIN)
def __push_model_to_hub(model: auto.transformers.AutoModel, /):
    model.push_to_hub(
        MY_REPOSITORY_NAME,
    )

adapter_model.bin:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

# Evaluate

In [4]:
@run(cond=DO_EVALUATE and not DO_LORA)
def model():
    return auto.transformers.AutoModelForCausalLM.from_pretrained(
        MY_REPOSITORY_NAME,
    )

@run(cond=DO_EVALUATE and DO_LORA)
def model():
    return auto.peft.PeftModel.from_pretrained(
        auto.transformers.AutoModelForCausalLM.from_pretrained(
            BASE_MODEL_NAME,
        ),
        MY_REPOSITORY_NAME,
    )

In [5]:
@run(cond=DO_EVALUATE)
def tokenizer():
    return auto.transformers.AutoTokenizer.from_pretrained(
        TOKENIZER_NAME,
        add_prefix_space=True,
    )

In [9]:
@run(cond=DO_EVALUATE)
def pipeline(model, tokenizer, /):
    return auto.transformers.pipeline(
        task='text-generation',
        model=model,
        tokenizer=tokenizer,
        device=0,
        max_new_tokens=256,
    )

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

In [10]:
@run(cond=False and DO_EVALUATE)
def __instructions():
    dataset = auto.datasets.load_dataset(
        MY_DATASET_NAME,
        split='train',
    )

    counter = auto.collections.Counter()
    def output(i, counter, /):
        auto.IPython.display.clear_output(wait=True)
        print(f'Processed {i:,d} datums')
        for inst, count in counter.most_common():
            print(auto.textwrap.fill(f'{count}: {inst}'))

    with auto.contextlib.suppress(KeyboardInterrupt):
        for i, datum in enumerate(dataset):
            if i != 0 and i % 50_000 == 0:
                output(i, counter)
            counter[datum['instruction']] += 1

    output(i, counter)

# 165175: You are an AI assistant. User will you give you a task. Your
# goal is to complete the task as faithfully as you can. While
# performing the task think step-by-step and justify your steps.
# 136285: You are a helpful assistant, who always provide explanation.
# Think like you are answering to a five year old.
# 110127: You are an AI assistant. You will be given a task. You must
# generate a detailed and long answer.
# 63267:
# 57303: You are an AI assistant that follows instruction extremely
# well. Help as much as you can.
# 51266: You are an AI assistant. Provide a detailed answer so user
# don’t need to search outside to understand the answer.
# 19146: You are an AI assistant that helps people find information.
# 18008: You are an AI assistant that helps people find information.
# User will you give you a question. Your task is to answer as
# faithfully as you can. While answering think step-bystep and justify
# your answer.
# 17181: You are an AI assistant that helps people find information.
# Provide a detailed answer so user don’t need to search outside to
# understand the answer.
# 9938: You should describe the task and explain your answer. While
# answering a multiple choice question, first output the correct
# answer(s). Then explain why other answers are wrong. Think like you
# are answering to a five year old.
# 8730: You are an AI assistant. You should describe the task and
# explain your answer. While answering a multiple choice question, first
# output the correct answer(s). Then explain why other answers are
# wrong. You might need to use additional knowledge to answer the
# question.
# 8599: Explain how you used the definition to come up with the answer.
# 8459: User will you give you a task with some instruction. Your job is
# follow the instructions as faithfully as you can. While answering
# think step-by-step and justify your answer.
# 7401: You are an AI assistant, who knows every language and how to
# translate one language to another. Given a task, you explain in simple
# steps what the task is asking, any guidelines that it provides. You
# solve the task and show how you used the guidelines to solve the task.
# 7212: You are a teacher. Given a task, you explain in simple steps
# what the task is asking, any guidelines it provides and how to use
# those guidelines to find the answer.
# 6372: Given a definition of a task and a sample input, break the
# definition into small parts. Each of those parts will have some
# instruction. Explain their meaning by showing an example that meets
# the criteria in the instruction. Use the following format: Part  # : a
# key part of the definition. Usage: Sample response that meets the
# criteria from the key part. Explain why you think it meets the
# criteria.
# 55: You are an AI assistant. Provide a detailed answer so user don't
# need to search outside to understand the answer.

In [11]:
@run(cond=DO_EVALUATE)
def completions(pipeline, /):
    PROMPTS = [
        __text(
            instruction="""You are a helpful AI assistant.""",
            input="""What is the capital of France?""",
        ),
        __text(
            instruction="""You are a helpful AI assistant.""",
            input="""What are the key features and characteristics of the Eiffel Tower?""",
        ),
        __text(
            instruction="""You are a helpful AI assistant.""",
            input="""What are the main causes and effects of climate change?""",
        ),
        __text(
            instruction="""You are a helpful AI assistant.""",
            input="""What were the major events and consequences of World War II?""",
        ),
    ]

    for prompt in PROMPTS:
        for model_name, pipeline in [
            (MY_REPOSITORY_NAME, pipeline),
            # (PEFT_MODEL, peft_pipeline),
        ]:
            for temperature in [
                # 0.25,
                0.50,
                # 0.75,
                1.00,
                # 1.25,
            ]:
                for trial in [
                    # None,
                    # None,
                    # None,
                    None,
                    None,
                ]:
                    completion = pipeline(prompt, do_sample=True, temperature=temperature)
                    completion = completion[0]
                    completion = completion['generated_text']

                    print(auto.json.dumps({
                        'prompt': prompt,
                        'model_name': model_name,
                        'temperature': temperature,
                        'completion': completion,
                    }))

                    with open('samples.json', 'at') as f:
                        print(auto.json.dumps({
                            'prompt': prompt,
                            'model_name': model_name,
                            'temperature': temperature,
                            'completion': completion,
                        }), file=f)

{"prompt": "<s>INSTRUCTION: You are a helpful AI assistant. INPUT: What is the capital of France? OUTPUT:", "model_name": "player1537/Bloom-560m-LoRA-trained-on-Dolphin", "temperature": 0.5, "completion": "<s>INSTRUCTION: You are a helpful AI assistant. INPUT: What is the capital of France? OUTPUT: The capital of France is the city of Paris. This city is located in the province of Paris, which is the largest city in France and the country's capital. It is the second-largest city in the world after London."}
{"prompt": "<s>INSTRUCTION: You are a helpful AI assistant. INPUT: What is the capital of France? OUTPUT:", "model_name": "player1537/Bloom-560m-LoRA-trained-on-Dolphin", "temperature": 0.5, "completion": "<s>INSTRUCTION: You are a helpful AI assistant. INPUT: What is the capital of France? OUTPUT: The capital of France is the city of Paris, which is located in the French department of Paris, France. It is the largest city in France, with a population of over 1.2 million people."}
{

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{"prompt": "<s>INSTRUCTION: You are a helpful AI assistant. INPUT: What are the main causes and effects of climate change? OUTPUT:", "model_name": "player1537/Bloom-560m-LoRA-trained-on-Dolphin", "temperature": 1.0, "completion": "<s>INSTRUCTION: You are a helpful AI assistant. INPUT: What are the main causes and effects of climate change? OUTPUT: The main causes and effects of climate change are:\n\n1. Increased precipitation is expected to increase in amount over time (in addition to increase in average precipitation; it also means increased total precipitation). \n2. Greater frequency of extreme precipitation events has also been reported.  Extreme weather conditions include precipitation outside of a range, rainfall exceeding the mean (a few inches), and extreme rainfall events.\n\nNow, let's look at the given event. The event was a rainy day with very high temperatures and high precipitation outside of the range, causing extreme weather conditions. Due to some natural disaster in 

In [18]:
@run
def __(pipeline, /):
    auto.pprint.pprint(pipeline(__text(
        instruction='You are an old country western with sage advice.',
        input='Give me advice that is appropriate for Tennessee on whiskey. What whiskey should I try?',
    ), temperature=1.5))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'generated_text': '<s>INSTRUCTION: You are an old country western with sage '
                    'advice. INPUT: Give me advice that is appropriate for '
                    'Tennessee on whiskey. What whiskey should I try? OUTPUT: '
                    'To find the right whiskey for Tennessee, you need to know '
                    'what the state has to offer. Here are some key factors '
                    'that you need to consider when choosing a whiskey:\n'
                    '\n'
                    '1. The state has a wide variety of different types of '
                    'whiskeys, including American, French, and German.\n'
                    '\n'
                    '2. The state has a wide variety of different types of '
                    'beers, including American, French, and German.\n'
                    '\n'
                    '3. The state has a wide variety of different types of '
                    'wine, including American, French, and German.\n'
        

# (end)